# Kurs 6

## I/O

In diesem Abschnitt wollen wir uns damit beschäftigen, wie wir in Julia Daten einlesen bzw. abspeichern können (input/output). Nehmen wir ab jetzt an, dass wir diesen DataFrame mit Daten speichern wollen:

In [ ]:
using DataFrames
df = DataFrame(
    "param1" => [1, 2],
    "param2" => [3, 4],
    "dicts" => [
        Dict(
            "d1" => 1.0,
            "d2" => 1.0
        ),
        Dict("d3" => 2.0)
    ]
)

### CSV

Das wohl einfachste Dateiformat ist CSV (comma-seperated values format). Hier haben wir letzen Endes nur eine Textdatei, in welcher wir einzelne Werte durch Kommata abtrennen.

In [ ]:
using Pkg; Pkg.add("CSV")
using CSV

In [ ]:
CSV.write("data.csv", df)

Oft wird anstelle des Kommas auch ein anderes Trennzeichen verwendet (zum Beispiel ein Semikolon). Wenn man nämlich wie im Deutschen Floats als `1,0`, `2,0`, etc. kodiert, dann braucht man logischerweise ein anderes Trennzeichen. Das geht wie folgt:

In [ ]:
CSV.write("data.csv", df, delim=";")

In [ ]:
read_csv = CSV.read("data.csv", DataFrame; delim=";")

Wie man aber hier schon sieht: Für geschachtelte Datenstrukturen ist CSV nicht besonders gut geeignet; unsere Dicts werden hier nur als String eingelesen:

In [ ]:
read_csv.dicts[1]

### Excel

Häufig trifft man natürlich auch auf Excel-Tabellen, dafür empfehlen wir das Paket XLSX.jl. Mehr dazu im Praxiskurs Julia.

### JSON

JSON steht für JavaScript Object Notation und ist ebenfalls leicht lesbar. Wir schreiben unsere Daten dabei wieder in ein Textfile; die Struktur ist allerdings nicht wie bei einer Tabelle. Diese ähnelt eher dem eines Dicts (also hat attribute-value Paare).

In [ ]:
Pkg.add("JSON"); using JSON

In [ ]:
stringdata = JSON.json(df)
println(stringdata)

In [ ]:
open("data.json", "w") do f
    write(f, stringdata)
end

In [ ]:
# create variable to write the information
read_json = Dict()
open("data.json", "r") do f
    global read_json
    dicttxt = read(f,String) # file information to string
    read_json = JSON.parse(dicttxt)  # parse and transform d ta
end
read_json

### JLD2

Dieses Package erlaubt uns Daten im HDF5-Format zu speichern, welches besser als etwa JSON für große Datenmengen geeignet ist. Bei JSON hat man da einige Defizite: Bei JSON wird jeder Wert durch Characters repräsentiert. Das bedeutet, der Float `3.141592653589793` wird mit 17 Zeichen gespeichert, wovon jedes in der Regel 8-Bit braucht (siehe [hier](https://de.wikipedia.org/wiki/American_Standard_Code_for_Information_Interchange)). Also speichern wir 136 anstelle der eigentlich benötigten 64 Bit! Zudem gibt es auch keine Kompression und der Zugriff auf Subdatensätze ist nicht performant. Bei HDF5 speichert man dagegen deutlich intelligenter; HDF5 ist nämlich ein Binärformat (binary format). Nähere Informationen findet man [hier](https://docs.hdfgroup.org/hdf5/develop/_intro_h_d_f5.html).

An dieser Stelle sei noch genannt, dass es alternative Pakete wie JLD.jl und HDF5.jl gibt, die je nach Anwendung besser geeignet sind.

In [ ]:
Pkg.add("JLD2"); using JLD2

In [ ]:
jldsave("df.jld2"; df)

In [ ]:
read_hdf5 = load("df.jld2")


### FileIO

Für viele Anwendungen müssen wir uns aber gar nicht mit diesen einzelnen Paketen rumschlagen, sondern brauchen nur das Paket `FileIO.jl` zu laden. Dieses stellt ein vereinheitlichtes Interface via `load` und `save` für zahlreiche Dateiformate bereit.

In [ ]:
Pkg.add("FileIO"); using FileIO

In [ ]:
x = collect(-3:0.1:3)
y = collect(-3:0.1:3)

xx = reshape([xi for xi in x for yj in y], length(y), length(x))
yy = reshape([yj for xi in x for yj in y], length(y), length(x))
                                
z = sin.(xx .+ yy.^2)

data_dict = Dict("x" => x, "y" => y, "z" => z)

save("data_dict.jld2", data_dict)

In [ ]:
read_hdf5 = load("data_dict.jld2")

x2 = read_hdf5["x"]
y2 = read_hdf5["y"]
z2 = read_hdf5["z"]

using Plots; plot(x2, y2, z2, st = :surface)

## Lineare Regression

Wir wollen in diesem Kurs nicht näher auf dieses Thema eingehen; der Vollständigkeit halber sei hier ein Minimalbeispiel für Lineare Regression.

In [ ]:
using Pkg; Pkg.add("GLM")

In [ ]:
using DataFrames, GLM
data = DataFrame(X=[1,2,3], Y=[2,4,7])

In [ ]:
fm = @formula(Y ~ X)
linear_regressor = lm(fm, data)

Wie man auf weitere Ergebnisse und Informationen zugreift, ist [hier](https://juliastats.org/GLM.jl/stable/) erklärt.

## Typen

Wir haben schon ganz am Anfang festgestellt, dass quasi alles, mit dem wir in Julia herumhantieren, einen Typ hat. Das ist aber nicht nur ein Nebenkriegsschauplatz, sondern faktisch super wichtig, weil wir uns auch eigene Typen (aka Datenstrukturen) und das dazugehörige Verhalten definieren können.[^1]

Grundsätzlich gibt es nur zwei Arten von Typen:

- abstract types: Können nicht instanziiert werden und haben keine Attribute (sind quasi Äste an Baum)
- concrete types: Können instanziiert werden, aber haben keine Subtypen (sind quasi Blätter an Baum).

Das wird an folgender Grafik ziemlich klar.

![Alt text](../graphics/types.png)

Alle blauen Felder sind *abstract types*, alles grünen Felder sind *concrete types*. Beispielsweise kann ein `Int64` initialisiert werden (einfach indem man etwa `a = 1` eingibt), ein `Real` kann das nicht. Dieser abstrakte Typ ist sozusagen lediglich der Kleber zwischen den verschiedenen reellwertigen Zahlen(sub-)typen. Dabei nennt man `Int64` einen *Subtyp* von `Real`; umgekehrt ist `Real` ein *Supertyp* von `Int64`. Weil ja wie gesagt in Julia alles aus Typen besteht, gilt es für diese nun einiges an Handwerkszeug zu erlernen.

[^1]: 
    Für die Leser mit etwas Vorwissen: Typen sind ähnlich zu structs in C, Klassen gibt es nicht. Im Gegensatz zu C definiert ein Typ aber tatsächlich einen neuen (benannten) Datentyp.

### Werkzeuge für Types

### Type declarations

Wichtig ist zunächst der `::`-Operator. Wir benutzen ihn vor allem für sanity checks (wir bekommen compile time anstelle von runtime errors) Spezialisierungen von Funktionen. Manchmal hilft er aber auch dem Compiler schnelleren Code zu produzieren (dazu später mehr).

In [ ]:
(1+2)::Float64

In [ ]:
(1+2)::Int

In [ ]:
foo::Int = 100.0
foo

Die Funktion `typeof` haben wir bereits gesehen und gibt uns den concrete type eines instanziierten Typs zurück:

In [ ]:
typeof("abc")

In [ ]:
isa("abc", AbstractString) # String ist ein Subtyp von AbstractString!

In [ ]:
isa(1, Float64) # ein Integer ist kein Float!

In [ ]:
isa(1.0, Float64)

In [ ]:
1.0 isa Number # alternative Syntax

In [ ]:
supertype(Int64) # der direkte (erste) Supertyp von Int64

In [ ]:
subtypes(Real) # direkte (erste) Subtypen des abstract types Real

In [ ]:
Int <: Real # <: checkt ob Typ ein Subtyp des rechten Typs ist

In [ ]:
Any # alle Objekte sind davon ein Subtyp; das ist quasi die Wurzel unseres Typenbaums

Ein kleine Sache von der man sich nicht verwirren lassen sollte: Wenn wir den Namen eines Typs eingeben, so wird dieser im Format `DataType` gehalten bzw. hat wiederum den Typ `DataType`:

In [ ]:
typeof(Int)

In [ ]:
typeof(DataType) # hier diesselbe Logik

In [ ]:
# wir schreiben uns gerade nochmal die typeof-Funktion
whichtype(::T) where T = T
whichtype("foo")

### Eigene Typen

Wir haben nun gesehen, wie wir mit bestehenden Typen umgehen können. Hier kommt nun der Teil, wo wir selbst kreativ werden können. Bevor wir zu dem sich eigentlich selbst erklärenden Beispiel kommen, sei noch gesagt: Es gibt verschiedene Arten von concrete types, ein `Int64` wäre beispielsweise ein *primitive type*. Für uns sind diese aber erstmal egal, viel wichtiger sind dagegen sogenannte *composite types*, die mit dem Keyword `struct` erstellt werden.

In [ ]:
abstract type Person end # abstract type

function fullname(p::Person) # type declaration in der Funktionssignatur; wir definieren Verhalten hier für einen abstrakten Typ!
    return "$(p.name) $(p.lastname)" # Zugriff auf Datenfelder via .
end

struct Student <: Person # composite type
    name::String # Feld
    lastname::String # Feld
    age::Int # Feld
    major::String # Feld
end

s = Student("Jane", "Doe", 22, "Computer Science")
fullname(s)

In [ ]:
s.name

In [ ]:
fullname(1) # geht nicht, weil für diesen Inputtypen unsere Funktion nicht definiert ist!

Wie wir sehen, können wir uns einen concrete type Student instanziieren, indem wir einfach die Funktion `Student` aufrufen und als Argumente die benötigten Datenfelder übergeben. Weil sozusagen aus dem Rezept `struct ... end` ein konkret lebender Wert in `s` erzeugt wird, nennt man die Funktion `Student` einen Konstruktor. Genauer gesagt, haben es wir hier mit dem *default constructor* zu tun, den Julia uns automatisch bereitstellt.

Oftmals wollen wir aber den Konstruktionsprozess einer Instanz modifizieren, weil wir zum Beispiel manche Felder mit default-Werten befüllen wollen. Betrachte dazu

In [ ]:
abstract type Equity end # Eigenkapital

struct Stock <: Equity # Aktie
    symbol::String
    name::String
end

struct StockQuantity # Anzahl einer Aktie, zum Beispiel im TradeRepublic-Konto
    stock::Stock
    quantity
end

my_stock = Stock("ADS", "Adidas")
StockQuantity(my_stock, 2)

Nun wollen wir sagen, dass wir per default eine Anzahl von 0 haben:

In [ ]:
StockQuantity(stock) = StockQuantity(stock, 0)
StockQuantity(my_stock)

Dieses Vorgehen nennt man einen äußeren Konstruktor, weil eben außerhalb der Typdefinition ein neues „Rezept“ auftaucht. Manchmal wollen wir aber auch den default constructor überschreiben, weil wir zum Beispiel den Benutzer vor einer sinnfreien Initialisierung schützen wollen. Dafür können wir einen *inner constructor* verwenden:

In [ ]:
const DAX_companies = ["SAP", "BASF", "Merck"]

struct SafeStock <: Equity # Aktie kann nur mit sinnvollen Werten initialisiert werden
    symbol::String
    name::String
    function SafeStock(symbol, name)
        if !(symbol in DAX_companies)
            println("$(symbol) ist keine bekannte AG!")
        else
            new(symbol, name)
        end
    end
end

SafeStock("DF", "d-fine")

In [ ]:
SafeStock("SAP", "SAP")

#### Parametrisierung

Bei unserem vorherigen Typ `StockQuantity` gibt es ein kleines Problem: Vielleicht wollen wir nun ein Feature in unserer Trading-App, die es erlaubt, auch nur Prozente einer Aktie zu halten. Dann müssten wir entweder einen neuen Typ schreiben, der dann ein Feld `quantity::Float64` hat (unpraktisch), oder wir lassen die type declaration weg und schreiben nur `quantity` (was äquivalent zu `quantity::Any` wäre). In letzterem Fall ist dann aber wiederum ungünstig, dass – wenn wir eine Variable vom Typ `StockQuantity` gegeben haben` – zur Kompilierzeit unklar ist, ob wir Prozente oder ganzzahlige Werte halten.

Deshalb gibt es für Typen noch eine nette Mechanik namens *Parametrisierung*:

In [ ]:
struct StockHolding{T<:Number}
    stock::Stock
    quantity::T
end

StockHolding(my_stock, 0.5)

#### Mutability

Instanzen eines structs sind *immutable*, das bedeutet: Datenfelder können nach der Instanziierung nicht mehr verändert werden!

In [ ]:
s.name = "Euler"

Das ist per se gut, denn wenn der Compiler weiß, dass sich nichts ändern kann, dann muss genau dafür während der Laufzeit nicht mehr gecheckt werden, sprich Code kann stärker optimiert werden. Je nach Anwendung ist das aber schon eine Funktionalität, die wir gerne hätten. Deshalb benutzt man dafür *mutable composite types*:

In [ ]:
mutable struct InsecureStudent <: Person # könnte Studiengang wechseln
    name::String
    major::String
end

s = InsecureStudent("John Doe", "WiMa")
s.major = "taxidriver"
s

### Unions

Der Union-Typ ist sehr nützlich, wenn wir verschiedene Datentypen kombinieren wollen, die aus verschiedenen Typhierarchien stammen.

In [ ]:
1 isa Union{Int, String}

In [ ]:
"1" isa Union{Int, String}

In [ ]:
abstract type Art end
struct Painting <: Art
    artist::String
    title::String
end

In [ ]:
struct BasketOfThings
    things::Vector{Union{Painting,Stock}}
    reason::String
end

In [ ]:
mona_lisa = Painting("Leonardo da Vinci", "Mona Lisa")
BasketOfThings([my_stock, mona_lisa], "Lehrpreis für das 3. OG")

In [ ]:
"1" isa Union{Int, String}